<a href="https://colab.research.google.com/github/skovz99/Exponential-Decay-for-Ordering-Time-Series-MLPRegression/blob/main/Exponential_Decay_for_Ordering_of_Time_Series_Inputs_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from itertools import product
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor

# import the stock ticker and price history
stock_ticker = yf.Ticker('TSLA')
price_history = yf.Ticker('TSLA').history(period='max', interval='1d', actions=False)
df = pd.DataFrame(price_history)

# define a function to implement exponential decay
def order_via_decay(inputs, size, decay):
    np_inputs = np.array(inputs)
    X_data = []
    Y_data = []
    for i in range(len(np_inputs) - size):
        X_window = np_inputs[i : i + size]
        y_value = np_inputs[i + size]
        X_data.append(X_window)
        Y_data.append(y_value)
    ordered_X_data = []
    for X_window in X_data:
        ordered_X_window = []
        for position, x in enumerate(X_window, start=1):
            order = x * (decay ** (size + 1 - position))
            ordered_X_window.append(order)
        ordered_X_data.append(ordered_X_window)
    return ordered_X_data, Y_data

# Set parameter ranges
input_size = [5, 10, 20, 25, 50] # this relates to the number of days in the past to consider. Any positive integer value works
test_sizes = [0.1, 0.2, 0.3] # this relates to the size of the testing dataset used for testing of the MLPRegression weights
decay_rate = [0.99, 0.98, 0.97, 0.95, 0.9, 0.8] # decay rate should be any number (0<x<1). Do not inlucde 0 or 1. Values present here are recommended values

# Create list of parameter combinations
param_combinations = product(test_sizes, input_size, decay_rate)

# Loop over parameter combinations and fit KNN model
test_scores1 = []
for test, input, decay in param_combinations:
  for x in range(3):
    X, Y = order_via_decay(df['Close'], size = input, decay = decay)
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = test, random_state=42, shuffle = False)
    architecture = MLPRegressor(hidden_layer_sizes=(100, 25, 10), activation = 'relu', max_iter = 100)
    predict = architecture.fit(X_train, y_train)
    y_pred = predict.predict(X_test)
    true_values = y_test
    predicted_values = y_pred
    check1 = pd.DataFrame([true_values, predicted_values])
    check1 = check1.T
    check1 = check1.rename(columns={0: "x", 1: "y"})
    check1['Difference'] = (abs((check1['x'] - check1['y']))/(check1['x']))*100 # instead of RMSE which will increase as the price of the stock goes up. Percentage difference between prediction and actual is used as metric of success
    mean1 = check1['Difference'].mean()
    ten = np.percentile(check1['Difference'], 10)
    twenty = np.percentile(check1['Difference'], 20)
    thirty = np.percentile(check1['Difference'], 30)
    fourty = np.percentile(check1['Difference'], 40)
    fifty = np.percentile(check1['Difference'], 50)
    sixty = np.percentile(check1['Difference'], 60)
    seventy = np.percentile(check1['Difference'], 70)
    eighty = np.percentile(check1['Difference'], 80)
    ninty = np.percentile(check1['Difference'], 90)
    test_scores1.append((mean1, ten, twenty, thirty, fourty, fifty, sixty, seventy, eighty, ninty))

test1 = pd.DataFrame(test_scores1, columns=['Mean', '10 Percentile', '20 Percentile', '30 Percentile', '40 Percentile', '50 Percentile', '60 Percentile', '70 Percentile', '80 Percentile', '90 Percentile'])
